In [1]:
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import random
from typing import List

In [2]:
samples = 500

m = 2
xs = [i * m for i in range(samples)]
c = 50

ms = [m for _ in range(samples)]
cs = [c for _ in range(samples)]
ys = [m * x + c for x in xs]
noise = [random.randint(0,50) for _ in range(samples)]

data = {
    'ms': ms,
    'xs': xs,
    'cs': cs,
    'noise': noise, 
    'ys': ys
}

df = pd.DataFrame(data=data)
df.head(5)

,ms,xs,cs,noise,ys
0,2,0,50,44,50
1,2,2,50,34,54
2,2,4,50,44,58
3,2,6,50,3,62
4,2,8,50,20,66


In [3]:
columns = ['ms', 'xs', 'cs', 'noise']
# columns = ['ms', 'xs', 'cs']

X_train = df[columns]
y_train = df['ys']

X_train.shape

(500, 4)

In [4]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=4, input_shape=[4]))
model.add(tf.keras.layers.Dense(units=1))
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='mean_squared_error')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 20        
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [5]:
epochs_hist = model.fit(X_train, y_train, epochs=500)

Epoch 1/500
16/16 [==============================] - 0s 6ms/step - loss: 549273.7500
Epoch 2/500
16/16 [==============================] - 0s 5ms/step - loss: 494712.9062
Epoch 3/500
16/16 [==============================] - 0s 5ms/step - loss: 442802.8750
Epoch 4/500
16/16 [==============================] - 0s 5ms/step - loss: 392213.2500
Epoch 5/500
16/16 [==============================] - 0s 5ms/step - loss: 344072.8125
Epoch 6/500
16/16 [==============================] - 0s 5ms/step - loss: 298431.5312
Epoch 7/500
16/16 [==============================] - 0s 5ms/step - loss: 255759.7031
Epoch 8/500
16/16 [==============================] - 0s 5ms/step - loss: 216546.0625
Epoch 9/500
16/16 [==============================] - 0s 5ms/step - loss: 181330.3125
Epoch 10/500
16/16 [==============================] - 0s 5ms/step - loss: 149411.7656
Epoch 11/500
16/16 [==============================] - 0s 5ms/step - loss: 121349.8672
Epoch 12/500
16/16 [==============================] - 0s 5ms/st

In [6]:
tdata = {
    'ms': [2,2,2,2],
    'xs': [0,2,4,6],
    'cs': [50,50,50,50],
    'noise': [123,384,2,23],
    'expected': [50,54,58,62]
}
test =  pd.DataFrame(tdata)
test

,ms,xs,cs,noise,expected
0,2,0,50,123,50
1,2,2,50,384,54
2,2,4,50,2,58
3,2,6,50,23,62


In [7]:
model_test = test[['ms', 'xs', 'cs', 'noise']]

model.predict(model_test)

1/1 [==============================] - 0s 47ms/step


array([[50.00065 ],
       [54.003113],
       [57.99952 ],
       [61.99971 ]], dtype=float32)

In [60]:
actual_weights = [model.layers[i].get_weights()[0] for i in range(len(model.layers))]
actual_biases = [model.layers[i].bias.numpy() for i in range(len(model.layers))]
weights = actual_weights.copy()
biases = actual_biases.copy()
output_weights = weights.pop(-1)
weights

[array([[ 0.3202289 ,  0.7401191 , -0.08377794, -0.04522135],
        [-0.39940256,  0.8308054 ,  0.37474367, -0.4668163 ],
        [-0.8989252 , -0.28096473, -0.13072267, -0.5290416 ],
        [-0.19297665, -0.4795451 , -0.03522943, -0.27107784]],
       dtype=float32)]

In [61]:
output_weights

array([[-0.6894979],
       [ 0.9847815],
       [ 0.7402753],
       [-1.347506 ]], dtype=float32)

In [62]:
def select_weights_first_layers(index: int) -> int:
    new_weights = []
    noises = []
    for layer in weights:
        for node in layer:
            noises.append(node[index])
            new_weights.append(np.delete(node, index))
    return (np.array(new_weights), np.array(noises))

In [63]:
new_weights, noises = select_weights_first_layers(3)

In [64]:
new_weights

array([[ 0.3202289 ,  0.7401191 , -0.08377794],
       [-0.39940256,  0.8308054 ,  0.37474367],
       [-0.8989252 , -0.28096473, -0.13072267],
       [-0.19297665, -0.4795451 , -0.03522943]], dtype=float32)

In [66]:
noises

array([-0.04522135, -0.4668163 , -0.5290416 , -0.27107784], dtype=float32)